In [1]:
import pandas as pd
from flot_util import *
from flot_visualization import *
from flot_prediction import *
from flot_feature_engineering import *

In [2]:
df = pd.read_csv(r'C:\Users\rcpsi\OneDrive\Documents\langchain\Flotation\dados\Flotacao_Dados_Final.csv',sep=';')

## Data cleaning

In [ ]:
df = parse_time(df)
df = remove_missing(df)
df = filter_by_ph(df, threshold=6)
#df = filter_by_flow(df)
df_clean = drop_consecutive_duplicates_tolerance(df)

#df_clean = add_time_features(df_clean)
#df_clean = add_ph_features(df_clean)
#df_clean = add_dosage_flow_features(df_clean)
#df_clean = add_cell_level_stats(df_clean)
#df_clean = add_rolling_features(df_clean)
df_clean = add_lag_silica_features(df_clean)

#df_clean = add_delta_silica_regular(df_clean)

In [4]:
df_clean['inicio'].max()

Timestamp('2025-01-31 23:00:00.002000')

In [5]:
#df_clean = filter_by_date_range(df_clean, '2024-07-18', '2024-10-20')

In [6]:
df_clean.shape

(2620, 24)

In [7]:
df_clean.head()

,inicio,fim,conc_silica,conc_fe,param_dosagem_amido,dosagem_amina_conc_magnetica,ph_flotacao_linha01,ph_flotacao_linha02,densidade_alimentacao_flotacao,nivel_celula_li640101,...,nivel_celula_li641102,nivel_celula_li641201,nivel_celula_li641202,nivel_celula_li642101,nivel_celula_li642201,nivel_celula_li643101,nivel_celula_li643201,vazao_alimentacao_flotacao,operacao,duration_min
0,2024-02-27 00:00:00,2024-02-27 00:20:00,2.72,64.59,492.50,152.54,9.47,7.99,1.60,369.44,...,350.88,344.09,349.76,360.64,359.21,360.89,328.20,907.14,True,20.0
1,2024-02-27 01:00:00,2024-02-27 01:20:00,1.23,67.40,492.50,147.58,9.60,7.95,1.57,369.84,...,353.15,340.23,342.37,360.43,348.20,361.36,332.58,677.98,True,20.0
2,2024-02-27 03:00:00,2024-02-27 03:20:00,1.93,65.09,491.67,135.00,9.50,7.92,1.60,368.38,...,352.21,346.62,347.69,360.32,365.79,360.09,340.50,848.19,True,20.0
3,2024-02-27 05:00:00,2024-02-27 05:20:00,2.04,65.46,491.67,146.53,9.38,9.50,1.60,369.72,...,350.05,338.10,341.43,358.81,362.28,359.22,365.90,873.97,True,20.0
4,2024-02-27 07:00:00,2024-02-27 07:20:00,4.56,63.75,491.67,150.75,9.43,9.52,1.60,370.38,...,345.31,358.72,347.50,356.60,355.33,356.71,333.77,922.34,True,20.0


## Exploratory data analysis

In [8]:
plot_time_series_with_gaps(df_clean,'inicio','conc_silica')

In [9]:
# Usage examples:
# plot_time_series(df_clean, 'inicio', 'conc_silica')
plot_histogram(df_clean, 'conc_silica')
# plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')
# plot_pairwise(df_clean, ['conc_silica', 'conc_fe', 'dosagem_amina_conc_magnetica', 'vazao_alimentacao_flotacao'])

In [10]:
# df_clean.columns

In [11]:
plot_scatter(df_clean, 'ph_flotacao_linha01', 'conc_silica')

In [12]:
plot_scatter(df_clean, 'vazao_alimentacao_flotacao', 'conc_silica')

## Prediction

In [13]:
df_clean.columns

Index(['inicio', 'fim', 'conc_silica', 'conc_fe', 'param_dosagem_amido',
       'dosagem_amina_conc_magnetica', 'ph_flotacao_linha01',
       'ph_flotacao_linha02', 'densidade_alimentacao_flotacao',
       'nivel_celula_li640101', 'TO_LI6401_02', 'nivel_celula_li640201',
       'nivel_celula_li640202', 'nivel_celula_li641101',
       'nivel_celula_li641102', 'nivel_celula_li641201',
       'nivel_celula_li641202', 'nivel_celula_li642101',
       'nivel_celula_li642201', 'nivel_celula_li643101',
       'nivel_celula_li643201', 'vazao_alimentacao_flotacao', 'operacao',
       'duration_min'],
      dtype='object')

In [14]:
features = df_clean.columns.tolist()
drops = ['inicio','fim','conc_fe', 'operacao','duration_min']
target = ['conc_silica']    

features = list(set(features) - set(drops))
features = list(set(features) - set(target))

print(df_clean.shape)
df_clean.dropna(inplace=True)
print(df_clean.shape)
df_clean.drop(columns=drops, inplace=True)


(2620, 24)
(2620, 24)


In [15]:
res_two = two_stage_quantile_then_conc(df_clean, features)
res_two

Stage 1 (Quantile) → MAE: 0.1976, MAPE: 52.90%, extremes: 52/524
Stage 2 (Silica) → MAE: 1.2320, MAPE: 39.53%


{'quantile_model': Pipeline(steps=[('scaler', StandardScaler()),
                 ('rf', RandomForestRegressor(n_estimators=200))]),
 'final_model': Pipeline(steps=[('scaler', StandardScaler()),
                 ('rf', RandomForestRegressor(n_estimators=200))]),
 'test_df':       conc_silica  param_dosagem_amido  dosagem_amina_conc_magnetica  \
 2380         1.54               487.01                        135.00   
 100          1.29               490.00                        135.00   
 902          1.91               498.77                        148.24   
 1253         2.68               490.00                        147.54   
 1987         1.61               497.33                        135.00   
 ...           ...                  ...                           ...   
 2310         0.71               492.26                        136.47   
 2284         3.19               495.48                        162.56   
 547          1.54               492.38                        140.54

In [23]:
res_two_per = two_stage_quantile_then_conc_percentiles(df_clean, features, show_residuals=True, test_size=0.4)
res_two_per

Stage 1 (Percentile) → MAE: 0.1977, MAPE: 157.71%, extremes: 14/1048
Stage 2 (Silica) → MAE: 1.3132, MAPE: 41.14%


{'percentile_model': Pipeline(steps=[('scaler', StandardScaler()),
                 ('rf', RandomForestRegressor(n_estimators=200))]),
 'final_model': Pipeline(steps=[('scaler', StandardScaler()),
                 ('rf', RandomForestRegressor(n_estimators=200))]),
 'test_df':       conc_silica  param_dosagem_amido  dosagem_amina_conc_magnetica  \
 442          1.65               490.00                        138.86   
 892          1.67               502.72                        167.48   
 2208         9.07               498.36                        151.85   
 226          1.25               495.13                        135.00   
 82           0.68               493.17                        150.00   
 ...           ...                  ...                           ...   
 2059         1.11               496.94                        135.00   
 862          2.00               491.00                        150.75   
 2344         1.72               485.82                        170.

In [17]:
res = evaluate_models(df_clean, features, target=target[0], show_residuals=False)
res

,model,MAE,RMSE,R2,MAPE (%)
0,LinearRegression,1.542683,2.629147,0.086427,70.106227
1,RandomForest,1.461500,2.554696,0.137434,63.496524
2,XGBoost,1.620073,2.629166,0.086414,71.527275


In [18]:
#res_weights = evaluate_models_weights(df_clean, features, target=target[0])
#res_weights

In [19]:
#res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.9)
#res_q

In [20]:
#res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.95)
#res_q

In [21]:
#res_q = evaluate_models_quantile(df_clean, features, target=target[0], show_residuals=False, quantile=0.5)
#res_q

In [22]:
#res_tweedie = evaluate_tweedie_models(df_clean, features, target=target[0], power=1.5,show_residuals=True)
#res_tweedie